# Meeting Report Parsing using Zoom API

## VCA Team, 4/13/2023

# Required Packages and Token

In [1]:
!pip install pyjwt
!pip install webvtt-py

In [2]:
import jwt
import requests
import json
from time import time
import os
import webvtt


In [3]:
# generate a token using the pyjwt library
token = jwt.encode(

    # Create a payload of the token containing
    # API Key & expiration time
    {'iss': API_KEY, 'exp': time() + 5000},

    # Secret used to generate token signature
    API_SEC,

    # Specify the hashing alg
    algorithm='HS256'
)

We have recorded a meeting with ID `85760048922` (updated 3/3/2023), and below we show how to retreive all the information after the meeting is over.

In [2]:
meeting_id = 85760048922

In [3]:
# input required from user
# For Sp23, use Dummy User Input

## DUMMY USER INPUT
meeting_title = "23min_demo_meeting"
meeting_date = '2023-03-03T19:59:40Z'
meeting_participants_count = 3
agenda_list = ["Duke and Duchess of Sussex asked to vacate UK home Frogmore Cottage", 
               "Alex Muradugh guilty",
               "Data Center Trends"]

In [4]:
# start setting up meeting_report_json
meeting_report_json = {}

In [ ]:
meeting_report_json['meeting_title'] = meeting_title


meeting_report_json['meeting_date'] = meeting_date

In [5]:
from datetime import datetime as dt
meeting_date

'2023-03-03T19:59:40Z'

In [44]:
# # Meeting details
# meet = requests.get(f'https://api.zoom.us/v2/past_meetings/'+str(meeting_id), headers=headers)
# if meet.status_code == 200:
#     response = json.loads(meet.text)
#     title = response['topic']
#     meeting_duration = response['total_minutes']

In [39]:
# Meeting participants
# as if start_time is 2023-03-03T19:59:40Z
start_time = '2023-03-03T19:59:40Z'
from datetime import datetime as dt
headers = {'authorization': 'Bearer %s' % token, 'content-type': 'application/json'}
part = requests.get(f'https://api.zoom.us/v2/past_meetings/'+str(meeting_id)+'/participants', headers=headers)
part_info = {}
if part.status_code == 200:
    response = json.loads(part.text)
    part_arr = response['participants']
    
    for p in part_arr:
        name = p['name']
        part_info[name] = {'late': 'late' if dt.strptime(p['join_time'], '%Y-%m-%dT%H:%M:%SZ') > dt.strptime(start_time, '%Y-%m-%dT%H:%M:%SZ') else 'on-time'}
        
# change dazhi's name in p_detail as it's in email :/
part_info['DAZHI PENG'] = part_info.pop('dazhip@andrew.cmu.edu')

# get speaking time
speaking_time = {}
for caption in webvtt.read(os.path.join('/Users/yooni/Desktop/vca_data', "85760048922_TRANSCRIPT.vtt")):
    message = caption.text.split(":")
    # get the participant ID and the speech text
    if len(message) == 1:
        participant_name = prev_speaker
    else:
        participant_name = message[0]
    prev_speaker = participant_name
    # calculate the speaking time for the participant
    time_format = "%H:%M:%S.%f"
    dt1 = dt.strptime(caption.start, time_format)
    dt2 = dt.strptime(caption.end, time_format)

    duration = (dt2 - dt1).total_seconds()
    if participant_name not in speaking_time:
        speaking_time[participant_name] = duration
    else:
        speaking_time[participant_name] += duration
speaking_time

{'Yooni Choi': 435.26,
 'Jiahe Feng': 497.6309999999999,
 'DAZHI PENG': 391.34100000000007}